Summary of Description:
- Supplies stored in marked crates, that are buried under crates -> rearrangement required
- ship has cargo crane to rearrange with fixed rearrangement order
- Input: starting stacks + rearrangement procedure
- Note: moving once at a time -> stack/pop

Problem:
- Output: which crate ends on top of each stack

Outline Idea:
- use stacks, do some popsicles
- function calls to parse line and process a given input

In [40]:
import sys
sys.path.append("..")
import lib
import re
import numpy as np

In [143]:
def parse_stack_line(line_array):
    craters = []
    for crater in line_array:
        if crater == 20:
            return craters
        else:
            craters.append(chr(crater))
    return craters

def parse_starting_stack(starting_stack):
    # split into stacks and numbers
    stacks, enumerators = re.split(r"\n(?=\s*\d)", starting_stack)

    # read number of stacks
    regex = re.compile(r"(\d+)\s*$") # capture last number
    n_stacks = int(regex.findall(enumerators)[0])
    print(f"n_stacks = {n_stacks}")

    # read matrix of stacks
    regex = re.compile(r"\s(\s)\s(?=$|\s{1}\[|\s{5}\[|\s{9}\[|\s{13}\[|\s{17}\[)|\[([A-Z])\]") # match all 3x whitespace or [A] and capture their middle
    content = regex.findall(stacks)
    content = np.array(content)[:,1] # only the captured group
    converter = lambda x: 20 if x == '' else ord(x)
    content = np.array(list(map(converter, content)))
    content = content.reshape((-1, n_stacks)) # get in shape
    content = np.flipud(content) # flip to get correct order for transpose
    content = content.T # now each line contains a stack in ASCII-numbering

    # parse down to obtain stacks
    stacks = [parse_stack_line(line_array) for line_array in content] # parse each line
    return stacks

def parse_moves(moving_order):
    regex = re.compile(r"move\s(\d+)\sfrom\s(\d+)\sto\s(\d+)")
    content = regex.findall(moving_order)
    content = np.array(content).flatten()
    content = np.array(list(map(int, content))) # convert to integers
    content = content.reshape((-1,3)) # each line has 3 numbers indicating move
    return content

def get_stack_and_moves(filename, verbose = False):
    with open(filename) as file:
        lines = file.read()

    # split stack and moves apart
    starting_stacks, moving_order = re.split(r"\n{2}", lines)
    starting_stacks = parse_starting_stack(starting_stacks)
    moving_order = parse_moves(moving_order)
    if verbose:
        print(f"Starting stack (n = {len(starting_stacks)}):\n{starting_stacks}")
        print(f"Moving order (m = {len(moving_order)}):\n{moving_order}")

    print(f"Parsing successful.")

    # now read moves
    return starting_stacks, moving_order

def process_stacks_cratemover9000(stacks, moving_order):
    for move in moving_order:
        n_repeats, source, goal = move
        source -= 1
        goal -= 1
        for i in range(n_repeats): # repeat move[0] times to move from stack move[1] to move[2]
            stacks[goal].append(stacks[source].pop())
    return stacks

def process_stacks_cratemover9001(stacks, moving_order):
    extra_stack = [] # as I'm lazy, repeat push/pop procedure twice via extra_stack to replicate stackmover 9001 behaviour
    for move in moving_order:
        n_repeats, source, goal = move
        source -= 1
        goal -= 1
        for i in range(n_repeats): 
            extra_stack.append(stacks[source].pop())
        for i in range(n_repeats):
            stacks[goal].append(extra_stack.pop())
    return stacks

In [140]:
stacks, moves = get_stack_and_moves("test_input.txt", True)
result = process_stacks_cratemover9000(stacks, moves)
print(result)

n_stacks = 3
Starting stack (n = 3):
[['Z', 'N'], ['M', 'C', 'D'], ['P']]
Moving order (m = 4):
[[1 2 1]
 [3 1 3]
 [2 2 1]
 [1 1 2]]
Parsing successful.
[['C'], ['M'], ['P', 'D', 'N', 'Z']]


In [142]:
stacks, moves = get_stack_and_moves("input.txt")
result = process_stacks_cratemover9000(stacks, moves)
print(result)

n_stacks = 9
Parsing successful.
[['P', 'L'], ['S', 'G', 'B', 'H', 'B', 'F', 'V', 'Z', 'D', 'R', 'B', 'W', 'J'], ['J', 'C', 'P', 'P', 'B', 'N', 'R', 'L', 'S'], ['C', 'F', 'V'], ['R', 'S', 'D', 'L'], ['T', 'T'], ['J', 'W'], ['J', 'T', 'V', 'C', 'D', 'B', 'M', 'R', 'N', 'W', 'P', 'H', 'B', 'B', 'T', 'R', 'P', 'Q'], ['N', 'P', 'M']]


Part A Result: L, J, S, V, L, T, W, Q, M

In [144]:
stacks, moves = get_stack_and_moves("test_input.txt", True)
result = process_stacks_cratemover9001(stacks, moves)
print(result)

n_stacks = 3
Starting stack (n = 3):
[['Z', 'N'], ['M', 'C', 'D'], ['P']]
Moving order (m = 4):
[[1 2 1]
 [3 1 3]
 [2 2 1]
 [1 1 2]]
Parsing successful.
[['M'], ['C'], ['P', 'Z', 'N', 'D']]


In [145]:
stacks, moves = get_stack_and_moves("input.txt")
result = process_stacks_cratemover9001(stacks, moves)
print(result)

n_stacks = 9
Parsing successful.
[['P', 'B'], ['T', 'Z', 'L', 'P', 'M', 'S', 'L', 'T', 'S', 'B', 'G', 'W', 'R'], ['R', 'H', 'B', 'L', 'C', 'T', 'J', 'H', 'Q'], ['N', 'R', 'W'], ['N', 'W', 'J', 'D'], ['R', 'B'], ['F', 'B'], ['V', 'P', 'T', 'C', 'J', 'D', 'B', 'V', 'C', 'S', 'P', 'R', 'N', 'V', 'F', 'B', 'D', 'J'], ['P', 'P', 'M']]


Part B Result: BRQWDBBJM